In [2]:
import os
import pickle
from langchain.docstore.document import Document as LangchainDocument
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from llama_parse import LlamaParse
import nest_asyncio
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.llm import LLMChain
from langchain.document_loaders import AssemblyAIAudioTranscriptLoader
import assemblyai as aai
from langchain.document_loaders import TextLoader
import json
from pathlib import Path
from pprint import pprint
import pytesseract
from PIL import Image
from moviepy.editor import VideoFileClip

In [3]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ""

llm_type = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = "sk-"
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-"
aai.settings.api_key = ""

# 1.0 Load Data

## 1.1 Load PDF Files

In [47]:
def save_to_file(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)

def load_from_file(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)
    
llama_documents_file = "../data/02_intermediate/llama_documents.pkl"
chroma_db_dir = "../data/03_primary/chroma_db"
nest_asyncio.apply()

try:
    llama_documents = load_from_file(llama_documents_file)
    print("Dados processados do PDF carregados com sucesso.")
except FileNotFoundError:
    # Configuração do LlamaParse
    parser = LlamaParse(
        result_type="text",  # Usar o método de texto
        num_workers=4,
        verbose=True,
        language="pt",
    )
    # Carregar e processar o documento PDF
    llama_documents = parser.load_data("../data/01_raw/Capítulo do Livro.pdf")

    save_to_file(llama_documents, llama_documents_file)
    print("Dados processados do PDF salvos com sucesso.")

structured_documents = [
    LangchainDocument(page_content=doc.text, metadata=doc.metadata) for doc in llama_documents
]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)

# Dividir os documentos em chunks menores
docs = text_splitter.split_documents(structured_documents)
for i, doc in enumerate(docs):
    doc.metadata["source"] = f"source_{i}"

Dados processados do PDF carregados com sucesso.


## 1.2 Load Text files

In [49]:
# Carregar arquivo de texto
text_loader = TextLoader("../data/01_raw/Apresentação.txt")
text_documents = text_loader.load()

# Processar e inserir no RAG
for doc in text_documents:
    doc.metadata["source"] = "Apresentação.txt"
docs.extend(text_documents)

## 1.3 Load JSON files

In [50]:
json_path="../data/01_raw/Exercícios.json"
data = json.loads(Path(json_path).read_text())
pprint(data)

{'_id': {'$oid': '5f1c22bbe64f7c00119f49fa'},
 'author': {'_id': {'$oid': '5f1b35cb0d339843d0f57542'},
            'alias': 'sagah',
            'name': 'SAGAH'},
 'banner': {'name': '296784bg.jpg',
            'size': 296784,
            'url': 'https://statics-marketplace.plataforma.grupoa.education/sagah/66bca21d-7024-450d-898c-58628b052366/ad54bf17-1d08-4f64-b50d-db0023d4a146.jpg'},
 'category': 'objective_exercise',
 'content': [{'_id': {'$oid': '5f1c22bbe64f7c00119f49f5'},
              'content': {'files': [],
                          'html': '<div '
                                  'class="question"><p></p><p><strong>Sobre a '
                                  'instrução &lt; !DOCTYPE html &gt; do HTML5, '
                                  'marque a alternativa correta: '
                                  '</strong></p><p></p></div>',
                          'options': [{'_id': 'c236f4c8-b692-4514-9bd5-fbd1d4ce0cae',
                                       'content': {'files

- Aqui vemos uma estrutura complexa do json, vamos extrair o que está em `content` e formatar referente as questões e os demais serão armazenados como metadado.

In [51]:
def process_json_data(data):
    processed_data = []
    metadata = {
        "_id": data["_id"]["$oid"],
        "external_id": data["external_id"],
        "name": data["name"],
        "external_topicId": data["external_topicId"],
        "title": data["title"],
        "type": data["type"],
        "language": data["language"],
        "author_name": data["author"]["name"],
        "author_alias": data["author"]["alias"],
        "tags": data["tags"],
        "banner_url": data["banner"]["url"],
        "created_at": data["created_at"]["$date"],
        "modified_at": data["modifed_at"]["$date"],
        "version": data["version"],
        "status": data["status"],
        "resource": data["resource"],
        "category": data["category"],
        "icon": data["icon"],
        "isReviewed": data["isReviewed"]
    }
    
    for question in data.get('content', []):
        question_metadata = metadata.copy()
        question_metadata.update({
            "question_id": question["_id"]["$oid"],
            "question_title": question.get("title"),
        })
        
        question_content = question.get("content", {})
        question_text = question_content.get("html", "")
        
        options = question_content.get("options", [])
        option_texts = []
        feedbacks = []
        correct_answers = []
        
        for option in options:
            option_text = option["content"].get("html", "")
            feedback = option["feedback"].get("html", "")
            correct = option.get("correct", False)
            
            option_texts.append(option_text)
            feedbacks.append(feedback)
            correct_answers.append(correct)
        
        combined_content = f"Pergunta: {question_text}\nOpções:\n" + \
                           "\n".join(f"- {opt}" for opt in option_texts) + \
                           "\nFeedbacks:\n" + \
                           "\n".join(f"- {feed}" for feed in feedbacks) + \
                           "\nCorretas:\n" + \
                           "\n".join(f"- {correct}" for correct in correct_answers)
        
        new_doc = LangchainDocument(page_content=combined_content, metadata=question_metadata)
        processed_data.append(new_doc)
    
    return processed_data

# Processar os dados JSON
processed_documents = process_json_data(data)

In [53]:
with open("../data/02_intermediate/processed_questions.json", "w") as f:
    json.dump([doc.dict() for doc in processed_documents], f)

# Carregar os documentos processados usando JSONLoader
json_loader = JSONLoader(
    file_path="../data/02_intermediate/processed_questions.json",
    jq_schema='.',
    text_content=False
)
json_documents = json_loader.load()

# Processar e inserir no RAG
for doc in json_documents:
    doc.metadata["source"] = "Exercícios.json"
docs.extend(json_documents)

## 1.4 Load Image files

- Para a imagem preciso de um modelo que faça reconhecimento de visão e transcreva para português.

In [62]:
def extract_text_from_image(image_path, lang='por'):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang=lang)
    return text

# Carregar e processar a imagem
image_path = "../data/01_raw/Infografico-1.jpg"
extracted_text = extract_text_from_image(image_path)

# Criar documento com o texto extraído
image_document = LangchainDocument(page_content=extracted_text, metadata={"source": "Infografico-1.jpg"})

# Processar e inserir no RAG
docs.append(image_document)

# Exibir o conteúdo extraído para verificação
print(image_document.page_content)

https://ww!

Contém informações que são apresentadas Governo do Brasil
no topo da página. Geralmente possui dados
da organização e os links mais gerais do site,
como página inicial, mapa do site, etc. PROGRAMA

REDUÇÃO D

Para ministros do «

NAV: incentivo irá estim

É uma área definida para os opa O ee

AÊ RE empreendedores.
principais atalhos da página.
Ações vão bene
50 mil crianças €

A intenção é afasta

Contém informações que são apresentadas crime e diminuir a
Z 2a e em toda a região
no rodapé da página. Normalmente tem EiGidE SRS

dados de contato da organização ou alguma
outra informação relevante.

Acesso à informação



- Não temos a melhor transcrição, mas é um ótimo passo para um modelo simples, uma opção robusta seria utilizar a API do chatgpt para visão computacional, mas como não tenho acesso não poderei incluir.

## 1.5 Load Video files

- Para o vídeo irei extrair apenas o áudio e converter para texto

In [64]:
video_file = "../data/01_raw/Dica_do_professor.mp4"
audio_file = "../data/02_intermediate/Dica_do_professor.mp3"

video = VideoFileClip(video_file)
video.audio.write_audiofile(audio_file)

In [65]:
# Carregar e transcrever o áudio
config = aai.TranscriptionConfig(language_code="pt")
audio_loader = AssemblyAIAudioTranscriptLoader(file_path=audio_file, config=config)
audio_documents = audio_loader.load()

# Processar e inserir no RAG
for doc in audio_documents:
    doc.metadata["source"] = "Dica do professor.mp4"
docs.extend(audio_documents)

## 1.6 Cleaning metadata

- Em especial a transcrição do áudio possui muitos metadados e vários como nulos, aqui irei tratá-los para um formato aceito pelo RAG

In [67]:
def find_and_correct_invalid_metadata(documents):
    problematic_docs = []
    for doc in documents:
        keys_to_remove = []
        for key, value in doc.metadata.items():
            if value is None or (isinstance(value, list) and len(value) == 0):
                problematic_docs.append((key, value, doc))
                # Corrigir metadados
                if value is None:
                    doc.metadata[key] = "unknown"
                elif isinstance(value, list) and len(value) == 0:
                    doc.metadata[key] = "empty_list"
            elif isinstance(value, list) and all(isinstance(item, dict) for item in value):
                keys_to_remove.append(key)

        for key in keys_to_remove:
            problematic_docs.append((key, doc.metadata[key], doc))
            del doc.metadata[key]

    return problematic_docs

# Localizar e corrigir documentos com metadados inválidos
problematic_docs = find_and_correct_invalid_metadata(docs)

# Exibir resultados
if problematic_docs:
    print("Documentos com metadados inválidos encontrados e corrigidos:")
    for key, value, doc in problematic_docs:
        print(f"Documento com {key} = {value} foi corrigido")
else:
    print("Nenhum documento com metadados inválidos encontrado.")

Documentos com metadados inválidos encontrados e corrigidos:
Documento com dual_channel = None foi corrigido
Documento com webhook_url = None foi corrigido
Documento com webhook_auth_header_name = None foi corrigido
Documento com webhook_auth_header_value = None foi corrigido
Documento com audio_start_from = None foi corrigido
Documento com audio_end_at = None foi corrigido
Documento com word_boost = [] foi corrigido
Documento com boost_param = None foi corrigido
Documento com redact_pii_audio_quality = None foi corrigido
Documento com redact_pii_policies = None foi corrigido
Documento com redact_pii_sub = None foi corrigido
Documento com speakers_expected = None foi corrigido
Documento com content_safety_confidence = None foi corrigido
Documento com custom_spelling = None foi corrigido
Documento com summary_model = None foi corrigido
Documento com summary_type = None foi corrigido
Documento com speech_threshold = None foi corrigido
Documento com speech_model = None foi corrigido
Docum

In [3]:
def load_from_file(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [4]:
compress_documents_files = "../data/03_primary/compress_documents.pkl"

save_to_file(docs, compress_documents_files)

docs = load_from_file(compress_documents_files)

# 2.0 Move data to RAG

- Quero anexar todos os dados tratados via Chromadb para que o modelo possa receber a informação e responder de acordo com o conteúdo.

In [ ]:
if os.path.exists(chroma_db_dir):
    docsearch = Chroma(persist_directory=chroma_db_dir, embedding_function=OpenAIEmbeddings(model="text-embedding-ada-002"))
    print("Chroma DB carregado com sucesso.")
else:
    # Indexar os documentos no Chroma
    docsearch = Chroma.from_documents(
        docs, OpenAIEmbeddings(model="text-embedding-ada-002"), persist_directory=chroma_db_dir
    )
    print("Chroma DB salvo com sucesso.")


docsearch.persist()
retriever = docsearch.as_retriever()

# 3.0 Test LLM with RAG

- Agora para o modelo quero validar como é o fluxo, utilizarei um prompt para simular um llm que recebe dúvidas, procura conteúdo disponível e responde.

In [86]:
llm = ChatOpenAI(model_name=llm_type, temperature=0)

prompt_template = PromptTemplate(template="""
Você é um assistente da empresa +A Educação onde vai receber usuários com dúvidas
sobre o conteúdo de estudo. Ajude os usuários indicando conteúdos relevantes
de acordo com o nível de dificuldade de cada um. Use exatamente o que estiver em: \n\n {document} \n\n
Caso não tenha a informação, informe que não possui informação sobre o tema
e sugira uma nova pergunta ao usuário.
Aqui está a questão do usuário: {question} \n

Responda de maneira clara e direta com base nas informações fornecidas.
""", input_variables=["question", "document"])

def format_docs(docs):
    formatted_docs = ""
    for i, doc in enumerate(docs):
        formatted_docs += f"Documento {i+1}:\n{doc.page_content}\n\n"
    return formatted_docs


retrieval_chain = LLMChain(prompt=prompt_template, llm=llm, output_parser=StrOutputParser())

def get_answer(question):
    # Recuperar os documentos relevantes
    retrieved_docs = retriever.get_relevant_documents(question)
    formatted_docs = format_docs(retrieved_docs)
    
    # Obter a resposta do modelo
    response = retrieval_chain.run(question=question, document=formatted_docs)
    return response, formatted_docs

In [87]:
question = "Qual foi a dica do professor sobre fundamentos de programação?"
answer, retrieved_docs = get_answer(question)
print("Resposta do Modelo:\n", answer)
print("\nDocumentos Recuperados:\n", retrieved_docs)

Resposta do Modelo:
 A dica do professor sobre fundamentos de programação foi sobre formatação de texto e âncoras em HTML, apresentando como utilizar tags específicas para formatar elementos visando o equilíbrio do código com a semântica do HTML5.

Documentos Recuperados:
 Documento 1:
Olá, seja bem-vindo à dica do professor da Unidade de Aprendizagem Criação de Páginas Web com HTML5. Neste vídeo, vamos falar sobre formatação de texto e âncoras em HTML. A especificação HTML5 disponibiliza uma série de elementos para formatar textos e criar âncoras em páginas web. Para isso, você deve usar tags específicas, que permitem formatar os elementos visando o equilíbrio do código com a semântica do HTML5. Nesta dica, criamos uma página HTML que apresenta uma relação de músicas de Roberto Carlos. Nesta página, montamos um menu de navegação que contém links para algumas páginas do Astro. O objetivo desses links é posicionar a página no início de cada música, sem precisar percorrer todo o document

In [88]:
question = "Qual é a questão 1 dos exercícios de fundamentos de programação?"
answer, retrieved_docs = get_answer(question)
print("Resposta do Modelo:\n", answer)
print("\nDocumentos Recuperados:\n", retrieved_docs)

Resposta do Modelo:
 A questão 1 dos exercícios de fundamentos de programação está relacionada às listas do HTML5. De acordo com o documento fornecido, as listas do HTML5 são desenvolvidas em uma página web com HTML5.

Documentos Recuperados:
 Documento 1:
<div class="question"><p></p><p><strong>Em relação às listas do HTML5, selecione a alternativa correta: </strong></p><p></p></div>

Documento 2:
<div class="question"><p></p><p><strong>Em relação às listas do HTML5, selecione a alternativa correta: </strong></p><p></p></div>

Documento 3:
DESENVOLVIMENTO
DE SISTEMAS PHP


Maurício de Oliveira Saraiva
---
Criação de páginas web
com HTML5


                                   Objetivos de aprendizagem


    Ao final deste texto, você deve apresentar os seguintes aprendizados:


       Definir a estrutura de uma página web com HTML5.
       Aplicar a formatação de texto em uma página web com HTML5.
       Desenvolver listas e tabelas em uma página web com HTML5.


Introdução

Document

In [90]:
question = "Quais são as alternativas da questão 1 dos exercícios de fundamentos de programação?"
answer, retrieved_docs = get_answer(question)
print("Resposta do Modelo:\n", answer)
print("\nDocumentos Recuperados:\n", retrieved_docs)

Resposta do Modelo:
 Desculpe, não possuo informações sobre as alternativas da questão 1 dos exercícios de fundamentos de programação. Você gostaria de saber mais sobre algum outro tema relacionado à programação? Posso te ajudar a encontrar informações sobre outros assuntos.

Documentos Recuperados:
 Documento 1:
<div class="question"><p></p><p><strong>Em relação às listas do HTML5, selecione a alternativa correta: </strong></p><p></p></div>

Documento 2:
<div class="question"><p></p><p><strong>Em relação às listas do HTML5, selecione a alternativa correta: </strong></p><p></p></div>

Documento 3:
<div class="question"><p></p><p><strong>Marque a alternativa correta sobre âncoras em HTML5: </strong></p><p></p></div>

Documento 4:
<div class="question"><p></p><p><strong>Marque a alternativa correta sobre âncoras em HTML5: </strong></p><p></p></div>




In [89]:
question = "Como posso aprender fundamentos de programação?"
answer, retrieved_docs = get_answer(question)
print("Resposta do Modelo:\n", answer)
print("\nDocumentos Recuperados:\n", retrieved_docs)

Resposta do Modelo:
 Para aprender fundamentos de programação, você pode começar estudando a criação de páginas web com HTML5. Este é um ótimo ponto de partida para entender a estrutura de páginas web, formatação de texto, listas e tabelas em HTML5. Bons estudos!

Documentos Recuperados:
 Documento 1:
Sistemas desenvolvidos para a plataforma web utilizam a linguagem HTML para a exibição de conteúdo, tanto para páginas estáticas como dinâmicas. Com HTML5 é possível criar páginas web com diversos recursos para a apresentação de dados por meio de novas marcações que permitem o uso de semântica e acessibilidade, facilitando a pesquisa por motores de busca automática e dispositivos próprios para deficientes visuais e auditivos.

Nesta Unidade de Aprendizagem, você vai estudar a estrutura de páginas web, a formatação de texto em documentos hipertexto e a apresentação de links, listas e tabelas em HTML5.

Bons estudos.

Ao final desta Unidade de Aprendizagem, você deve apresentar os seguintes

- No final temos um fluxo completo do modelo, considero esse o primeiro ciclo do projeto como concluido, a partir dessa etapa irei desenvolver o modelo via api e disponibilizar via uma interface de testes, também irei melhorar o fluxo de prompt e interação.